<a href="https://colab.research.google.com/github/akilasarva/jaxa_work/blob/main/best_spots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install gurobipy
%matplotlib inline
import random

import gurobipy as gp
from gurobipy import GRB

import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import MiniBatchKMeans

from PIL import Image, ImageDraw
import numpy as np
import cv2
import os
import math
from keras.preprocessing.image import save_img

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 18.4 MB/s 


ImportError: ignored

In [ ]:
def find_color(f, im, mincolor, maxcolor, colorname):
  hsv_img = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
  COLOR_MIN = np.array(mincolor,np.uint8)
  COLOR_MAX = np.array(maxcolor,np.uint8) 
  mask = cv2.inRange(np.float32(hsv_img), COLOR_MIN, COLOR_MAX)

  imask = mask>0
  color = np.zeros_like(im, np.uint8)
  color[imask] = im[imask]
  coords = np.array(np.where(imask)).T
  color = cv2.resize(color, (700, 700))
  cv2.imwrite(colorname + '_' + f, color)
  return list(coords)

def dist(loc1, loc2):
    return np.linalg.norm(loc1-loc2, ord=2) # Euclidean distance

def get_locs(f):
  img = cv2.imread(f)
  w,h,_ = img.shape
  o_coords = find_color(f, img, [10, 50, 50], [22, 255, 255], 'orange')
  r_coords = find_color(f, img, [0, 100, 20], [9, 255, 255], 'red')
  g_coords = find_color(f, img, [35, 90, 90], [85, 255, 255], 'green')

  customer_locs = [*g_coords, *o_coords, *o_coords, *r_coords, *r_coords, *r_coords]
  facility_locs = []

  for i in range(5):
    for j in range(5):
      facility_locs += [(h*1.0*i/5 + h/10, w*1.0*j/5 + w/10)]
  return customer_locs, facility_locs

def kmean(num_clusters, seed, customer_locs):
  kmeans = MiniBatchKMeans(n_clusters=num_clusters, init_size=3*num_clusters, random_state=seed).fit(customer_locs)
  memberships = list(kmeans.labels_)
  centroids = list(kmeans.cluster_centers_) #center point for each cluster
  weights = list(np.histogram(memberships, bins=num_clusters)[0]) #number of customers in each cluster
  
  return centroids, weights

def plots(customer_locs, centroids, facility_locs, pairings, f):
  num = 7
  plt.figure(figsize=(7,7))
  plt.axis('off')
  plt.scatter(*zip(*customer_locs), c='Pink', s=0.2)
  plt.scatter(*zip(*centroids), c='Red', s=10)
  plt.scatter(*zip(*facility_locs), c='Green', s=10)
  ax = plt.gca() #you first need to get the axis handle
  ax.set_aspect(1)

  assignments = [p for p in pairings if assign[p].x > 0.5]
  fac_assign = []
  for p in assignments:
      pts = [facility_locs[p[0]], centroids[p[1]]]
      if facility_locs[p[0]] not in fac_assign:
        fac_assign.append(facility_locs[p[0]])
      plt.plot(*zip(*pts), c='Black', linewidth=0.5)
  plt.scatter(*zip(*fac_assign), s=threshold*num*9, alpha=0.2, c = 'blue')
  plt.savefig('output_'+f)
  plt.clf()

In [ ]:
seed = 10101
num_candidates = 25
max_facilities = 4
num_clusters = 25
threshold = 400

fs = ['4pm.png', '8am.png','12pm.png', '230pm.png', '530pm.png', '10pm.png']

for f in fs:
  customer_locs, facility_locs = get_locs(f)
  centroids, weights = kmean(num_clusters, seed, customer_locs)

  pairings = {(facility, cluster): dist(facility_locs[facility], centroids[cluster])
            for facility in range(num_candidates)
            for cluster in range(num_clusters) 
            if  dist(facility_locs[facility], centroids[cluster]) < threshold}

  print("Number of viable pairings: {0}".format(len(pairings.keys())))

  m = gp.Model("Facility location")
  # Decision variables: select facility locations
  select = m.addVars(range(num_candidates), vtype=GRB.BINARY, name='select')
  # Decision variables: assign customer clusters to a facility location
  assign = m.addVars(pairings.keys(), vtype=GRB.BINARY, name='assign')

  # Deploy Objective Function
  # 0. Total distance
  obj = gp.quicksum(weights[cluster]
                *pairings[facility, cluster]
                *assign[facility, cluster]
                for facility, cluster in pairings.keys())
  m.setObjective(obj, GRB.MINIMIZE)

  # 1. Facility limit
  m.addConstr(select.sum() <= max_facilities, name="Facility_limit")

  # 1b. Proximity constraints
  #m.addConstr(select)

  # 2. Open to assign
  m.addConstrs((assign[facility, cluster] <= select[facility]
              for facility, cluster in pairings.keys()),
              name="Open2assign")

  # 3. Closest store
  m.addConstrs((assign.sum('*', cluster) == 1
              for cluster in range(num_clusters)),
              name="Closest_store")

  # Find the optimal solution
  m.optimize()  

  plots(customer_locs, centroids, facility_locs, pairings, f)

Number of viable pairings: 298
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 324 rows, 323 columns and 919 nonzeros
Model fingerprint: 0x973bae8f
Variable types: 0 continuous, 323 integer (323 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.00s
Presolved: 324 rows, 323 columns, 919 nonzeros
Variable types: 0 continuous, 323 integer (323 binary)
Found heuristic solution: objective 4.353226e+07

Root relaxation: objective 2.937089e+07, 173 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2.937089e+07 2.9371e+07  0.00%     -    0s

Explored 1 nodes (173 simplex iterations)

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

In [ ]:
num =7
plt.figure(figsize=(num,num))
plt.axis('off')
plt.scatter(*zip(*customer_locs), c='Pink', s=0.2)
plt.scatter(*zip(*facility_locs), c='Green', s=10)
ax = plt.gca() #you first need to get the axis handle
ax.set_aspect(1)
print(facility_locs)
fac_assign = [(253, 230), (590, 230), (253, 536), (590,536)]
plt.scatter(*zip(*fac_assign), s=threshold*num*7, alpha=0.2, c = 'blue')
plt.savefig('output.png')
plt.clf()

[(84.2, 76.5), (84.2, 229.5), (84.2, 382.5), (84.2, 535.5), (84.2, 688.5), (252.60000000000002, 76.5), (252.60000000000002, 229.5), (252.60000000000002, 382.5), (252.60000000000002, 535.5), (252.60000000000002, 688.5), (421.0, 76.5), (421.0, 229.5), (421.0, 382.5), (421.0, 535.5), (421.0, 688.5), (589.4, 76.5), (589.4, 229.5), (589.4, 382.5), (589.4, 535.5), (589.4, 688.5), (757.8000000000001, 76.5), (757.8000000000001, 229.5), (757.8000000000001, 382.5), (757.8000000000001, 535.5), (757.8000000000001, 688.5)]


<Figure size 504x504 with 0 Axes>